In [1]:
from main import DocCorpus
from corpus import Corpus
import ipywidgets as widgets
from IPython.display import display
from IPython.display import HTML
from datetime import datetime
import os

In [13]:
c = Corpus("Mon Corpus")
choix_action = widgets.RadioButtons(options=['Créer un corpus', 'Charger un corpus'], description='Que voulez-vous faire ? ')
sujet_corpus = widgets.Text(description='Sujet du corpus :', disabled=False)
nombre_articles = widgets.IntText(value=10, description='Nombre d\'articles :', disabled=True)
# Liste déroulante des corpus déjà disponibles, on ampute l'extension .pkl
fichiers_pkl = [f[:-4] for f in os.listdir("all_corpus") if f.endswith(".pkl")]
chemin_corpus = widgets.Dropdown(options=fichiers_pkl,description='Sujet :',disabled=True,)
# Conteneur pour regrouper les widgets
conteneur_widgets = widgets.VBox([choix_action, sujet_corpus, nombre_articles, chemin_corpus])
bouton_executer = widgets.Button(description='Exécuter')

def on_choix_change(change,choix_action,sujet_corpus,nombre_articles,chemin_corpus):
    if change['type'] == 'change' and change['name'] == 'value':
        if choix_action.value == 'Créer un corpus':
            sujet_corpus.disabled = False
            nombre_articles.disabled = False
            chemin_corpus.disabled = True
        elif choix_action.value == 'Charger un corpus':
            sujet_corpus.disabled = True
            nombre_articles.disabled = True
            chemin_corpus.disabled = False

def update_dropdown():  # Mettre à jour la liste déroulante des corpus disponibles
    fichiers_pkl = [f[:-4] for f in os.listdir("all_corpus") if f.endswith(".pkl")]
    chemin_corpus.options = fichiers_pkl

def charger_corpus():
    global query
    global c
    if choix_action.value == 'Créer un corpus':
        query = sujet_corpus.value
        nArticle = nombre_articles.value
        DocCorpus(query, nArticle)
    elif choix_action.value == 'Charger un corpus':
        query = chemin_corpus.value  
    c = c.load("all_corpus/" + query +'.pkl')
    init = c.construire_vocabulaire()

def on_bouton_executer_click(b):
    charger_corpus()
    update_dropdown()

# On observe les boutons radio pour déclencher la fonction on_choix_change
choix_action.observe(lambda change: on_choix_change(change, choix_action, sujet_corpus, nombre_articles, chemin_corpus), names='value')
bouton_executer.on_click(on_bouton_executer_click)

display(conteneur_widgets)
display(bouton_executer)


Button(description='Exécuter', style=ButtonStyle())

In [14]:
qWidget = widgets.Text(value='', placeholder='Entrez des mots-clés :')
auteurWidget = widgets.Text(value='', placeholder='Entrez l\'auteur :')
sourceDropdown = widgets.Dropdown(
    options=['Toutes les sources', 'Reddit', 'Arxiv'],
    value='Toutes les sources',
    description='Source:',
)
datePicker = widgets.DatePicker(
    description='Sélectionnez une date',
    disabled=False
)
button = widgets.Button(description='Recherche')
output = widgets.Output()

def recherche_et_affichage(keywords, auteur, source, selected_date):
    res_recherche = c.recherche(keywords)
    html_output = ""
    for resultat in res_recherche.items():
        index_doc = resultat[0]
        doc = c.id2doc[index_doc]

        if (source == 'Toutes les sources' or doc.type == source):
            # Convertir la date en objet datetime au format 'YYYY/MM/DD'
            doc_date = datetime.strptime(doc.date, '%Y/%m/%d').date()
            # Comparer avec la date sélectionnée et l'auteur
            if (selected_date is None or doc_date == selected_date) and (auteur.lower() in doc.auteur.lower()):
                # Formate la date dans le format 'YYYY/MM/DD'
                formatted_date = doc_date.strftime('%Y/%m/%d')
                html_output += f"<p><b>Document : </b> {doc.titre}</p>"
                html_output += f"<p><b>Auteur : </b> {doc.auteur}</p>"
                html_output += f"<p><b>Date : </b> {formatted_date}</p>"
                html_output += f"<p><b>Source : </b> {doc.type}</p>"
                html_output += f"<p><b>Contenu : </b> {doc.texte}</p>"
                html_output += f"<p><b>URL : </b><a href='{doc.url}' target='_blank'> {doc.url}</a></p>"
                html_output += "=" * 50 + "<br><br>"

    with output:
        output.clear_output(wait=True)
        display(HTML(html_output))

def on_button_click(b):
    output.clear_output()
    recherche_et_affichage(qWidget.value, auteurWidget.value, sourceDropdown.value, datePicker.value)
    c.__repr__

button.on_click(on_button_click)
display(qWidget, auteurWidget, sourceDropdown, datePicker, button, output)

Text(value='', placeholder='Entrez des mots-clés :')

Text(value='', placeholder="Entrez l'auteur :")

Dropdown(description='Source:', options=('Toutes les sources', 'Reddit', 'Arxiv'), value='Toutes les sources')

DatePicker(value=None, description='Sélectionnez une date', step=1)

Button(description='Recherche', style=ButtonStyle())

Output()

c:\Users\theap\anaconda3\Lib\site-packages\scipy\sparse\_index.py:103: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])
